# Numba

---

A Just-In-Time (JIT) compiler is a compiler that runs while your program is running, instead of doing all the compilation work up front. It takes intermediate code (often bytecode) and, when a piece of code is about to be executed a lot, it compiles that specific part into native machine instructions for the current CPU. The compiled machine code is then reused for subsequent executions, so the hot parts of the program can run much faster than if they were interpreted every time.

There are two main alternatives to JIT compilation. One is pure interpretation, where the runtime reads bytecode (or source) and executes it instruction by instruction without ever generating native machine code; this keeps startup simple but usually leaves more performance on the table. The other is ahead-of-time (AOT) compilation, where all or most code is compiled to machine code before the program starts, like typical C/C++ binaries; this removes JIT overhead and often improves startup time, but it cannot use as much runtime information to specialize and optimize code as a JIT can. In practice, many modern runtimes blend these ideas: they may interpret at first, then JIT-compile only the hot paths to get a good balance of startup speed and long-run performance.


In [1]:
import numba as nb
import numpy as np
from scipy.stats import norm

from theoria.validor import TestCase, Validor

if __name__ == "__main__":
    np.random.seed(42)

# Mechanism

Numba is a JIT compiler that builds on LLVM: when you call a `@njit` function, Numba takes the Python bytecode and argument types, lowers them into its own typed intermediate form, and lets LLVM handle heavy-duty optimization and native code generation for your CPU. LLVM itself is a language-agnostic compiler infrastructure: it defines a common intermediate representation (LLVM IR) plus a rich set of optimization and codegen passes, so many different languages (C/C++, Rust, Swift, and tools like Numba) can share the same optimization pipeline and backends to produce efficient machine code for multiple architectures.

**The compiled version is cached**, so subsequent calls with the same argument types jump directly into machine code instead of going through the Python interpreter, which eliminates most of the usual overhead from Python’s dynamic dispatch and object boxing.



## Best Practices


Numba is not limited to NumPy, but NumPy arrays are where it works best. It can compile pure-Python numerical code that uses basic types (ints, floats, bools), loops, if/else logic, and a subset of containers, as long as you stay within the features it supports in “nopython” mode. However, NumPy arrays fit Numba’s model extremely well: they are homogeneous, contiguous blocks of memory with well-defined dtypes, so Numba can lower operations on them to efficient pointer arithmetic and vectorized machine instructions. Numba also knows how to generate native code for many NumPy operations and ufuncs instead of calling back into Python, which is why combining NumPy + Numba often gives large speedups on array-heavy workloads with complex loops or per-element logic.

## Example: Vanilla Call Option Pricing

In [2]:
def price_numpy(
    S0: float,
    K: float,
    T: float,
    r: float,
    sigma: float,
    n_simulations: int,
) -> float:
    discount = np.exp(-r * T)
    drift = (r - 0.5 * sigma * sigma) * T
    diffusion = sigma * np.sqrt(T)

    W = np.random.standard_normal(n_simulations)
    ST = S0 * np.exp(drift + diffusion * W)
    return discount * np.mean(np.maximum(ST - K, 0))


@nb.njit(parallel=True, fastmath=True)
def price_numba(
    S0: float,
    K: float,
    T: float,
    r: float,
    sigma: float,
    n_simulations: int,
) -> float:
    discount = np.exp(-r * T)
    drift = (r - 0.5 * sigma * sigma) * T
    diffusion = sigma * np.sqrt(T)

    payoffs = 0.0

    for _ in nb.prange(n_simulations):
        W = np.random.normal()
        ST = S0 * np.exp(drift + diffusion * W)
        payoffs += max(ST - K, 0)

    return discount * payoffs / n_simulations


def price_analytical(
    S0: float,
    K: float,
    T: float,
    r: float,
    sigma: float,
) -> float:
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    call_price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

In [3]:
%%timeit -n 10 -r 1

# Vectorized NumPy pricing
numpy_price = price_numpy(
    S0=100,
    K=100,
    T=1,
    r=0.05,
    sigma=0.2,
    n_simulations=10_000_000,
)

319 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


In [4]:
%%timeit -n 10 -r 1

# Numba-optimized pricing
numba_price = price_numba(
    S0=100,
    K=100,
    T=1,
    r=0.05,
    sigma=0.2,
    n_simulations=10_000_000,
)

220 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


In [5]:
%%timeit -n 10 -r 1

# Analytical pricing
analytical_price = price_analytical(
    S0=100,
    K=100,
    T=1,
    r=0.05,
    sigma=0.2,
)

250 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


In [6]:
num_iterations = 100_000_000


def comparison(
    actual_price: float,
    expected_price: float,
    rel_tol: float = 1e-3,
) -> bool:
    return abs((actual_price - expected_price) / expected_price) < rel_tol


test_cases = [
    TestCase(
        input_data={
            "S0": 100,
            "K": 100,
            "T": 1,
            "r": 0.05,
            "sigma": 0.2,
            "n_simulations": num_iterations,
        },
        expected_output=price_analytical(
            S0=100,
            K=100,
            T=1,
            r=0.05,
            sigma=0.2,
        ),
        description="MC price should approximate analytical price",
    ),
    TestCase(
        input_data={
            "S0": 150,
            "K": 100,
            "T": 0.5,
            "r": 0.03,
            "sigma": 0.25,
            "n_simulations": num_iterations,
        },
        expected_output=price_analytical(
            S0=150,
            K=100,
            T=0.5,
            r=0.03,
            sigma=0.25,
        ),
        description="MC price should approximate analytical price for ITM option",
    ),
    TestCase(
        input_data={
            "S0": 80,
            "K": 100,
            "T": 2,
            "r": 0.04,
            "sigma": 0.3,
            "n_simulations": num_iterations,
        },
        expected_output=price_analytical(
            S0=80,
            K=100,
            T=2,
            r=0.04,
            sigma=0.3,
        ),
        description="MC price should approximate analytical price for OTM option",
    ),
]

Validor(price_numpy).add_cases(test_cases).run(comparison)
Validor(price_numba).add_cases(test_cases).run(comparison)

[2026-01-16 12:25:07,165] [INFO] All 3 tests passed for price_numpy.


[2026-01-16 12:25:10,178] [INFO] All 3 tests passed for price_numba.
